# Subject level parcellation
This juypter pipeline is used to perform a parcelllation on the subject level.
Each subject can i their personalized parcellation using their resting state.



In [ ]:
import numpy as np
import networkx as nx
import nibabel as nib
from pathlib import Path

from parcellation_surface.preprocessing_surface import load_data_normalized
from parcellation_surface.similarity_matrix import compute_similarity_matrix_pca
from parcellation_surface.smoothing import smooth_surface_graph
from parcellation_surface.gradient import compute_gradients, build_mesh_graph
from parcellation_surface.watershed import watershed_by_flooding
from parcellation_surface.visualization import visualize_brain_surface

### This cell follows the literature method, but is time-consuming

In [ ]:
# Full pipeline subject level
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Start modification 
config = {
"fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
"subjects_dir": Path(r"D:\Data_Conn_Preproc\PPSFACE_N18")
}
run = 1
subject = f"{2:02d}" #TODO : choose the subject number
subj_dir = config["subjects_dir"] / f"sub-{subject}"
# Output paths
output_dir = subj_dir / f"sub-{subject}_parcellation"
(output_dir).mkdir(exist_ok=True, parents=True)
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>> end modification

for hemisphere in ['lh', 'rh']:
    #TODO: remove this line afterwards
    if hemisphere == 'rh':
       continue
    print(f"\n\n\nProcessing subject {subject} hemisphere {hemisphere}\n\n\n")
    
    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Start modification 
    # Surface paths
    surface_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.white"
    surface_inf_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.inflated"
    # Fmri paths
    surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
    vol_fmri_path = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
    brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"
    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>> end modification

    # Extract the Surface Mesh
    coords, faces = nib.freesurfer.read_geometry(str(surface_path))
    coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
    graph = build_mesh_graph(faces)

    # Load the data
    surf_fmri_n, vol_fmri_n = load_data_normalized(surf_fmri_path,
                                                    vol_fmri_path, 
                                                    brain_mask_path)
    # 1 : Compute the similarity matrix
    print('Computing similarity matrix')
    sim_matrix = compute_similarity_matrix_pca(vol_fmri_n,
                                               surf_fmri_n)

    # Save the similarty matrix
    # sim_matrix_path = output_dir / f"sim_matrix_sub{subject}_run{run}_{hemisphere}.npy"
    # np.save(sim_matrix_path, sim_matrix)
    
        
    # 2 : Smooth the similatry matrix
    print('Smoothing similarity matrix')
    sim_matrix_smoothed = smooth_surface_graph(graph, sim_matrix, iterations=5)

    # 3 : Compute the gradient of the similarty matrix
    print('Computing gradients')
    gradients = compute_gradients(graph, sim_matrix_smoothed)

    # 3.1 : Smooth the gradient
    print('Smoothing gradients')
    gradients_smoothed = smooth_surface_graph(graph, gradients, iterations=10)
    gradients_path = output_dir / f"gradients_smoothed_sub{subject}_run{run}_{hemisphere}.npy"
    np.save(gradients_path, gradients_smoothed)

    # 4 : Create the boundary map from the watershed
    boundary_map = np.zeros_like(gradients[:,0]).astype(np.float32)
    for map_idx in range(gradients.shape[1]): # loop over each columns of the matrix
        if map_idx % 10 == 0:
            print(f"Processing map {map_idx}") # Print the current map index
        boundary = (watershed_by_flooding(graph, gradients_smoothed[:,map_idx])<0)*1 # Extract the bounary from the watershed algorithm
        boundary_map += boundary
    
    boundary_map_path = output_dir / f"boundary_map_sub{subject}_run{run}_{hemisphere}.npy"
    np.save(boundary_map_path, boundary_map)
    # memory clean up
    del boundary_map, gradients, gradients_smoothed, sim_matrix, sim_matrix_smoothed

### This Cell is faster but doesn't follow the litterature

In [ ]:
# Full pipeline subject level
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Start modification 
config = {
"fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
"subjects_dir": Path(r"D:\Data_Conn_Preproc\PPSFACE_N18")
}
run = 1
subject = f"{10:02d}" #TODO : choose the subject number
subj_dir = config["subjects_dir"] / f"sub-{subject}"
# Output paths
output_dir = subj_dir / f"sub-{subject}_parcellation"
(output_dir).mkdir(exist_ok=True, parents=True)
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>> end modification

for hemisphere in ['lh', 'rh']:

    print(f"\n\n\nProcessing subject {subject} hemisphere {hemisphere}\n\n\n")
    
    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Start modification 
    # Surface paths
    surface_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.white"
    surface_inf_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.inflated"
    # Fmri paths
    surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
    vol_fmri_path = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
    brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"
    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>> end modification

    # Extract the Surface Mesh
    coords, faces = nib.freesurfer.read_geometry(str(surface_path))
    coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
    graph = build_mesh_graph(faces)

    # Load the data
    surf_fmri_n, vol_fmri_n = load_data_normalized(surf_fmri_path,
                                                    vol_fmri_path, 
                                                    brain_mask_path)
    # 1 : Compute the similarity matrix
    print('Computing similarity matrix')
    sim_matrix = compute_similarity_matrix_pca(vol_fmri_n,
                                               surf_fmri_n)

    # Save the similarty matrix
    # sim_matrix_path = output_dir / f"sim_matrix_sub{subject}_fastway_run{run}_{hemisphere}.npy"
    # np.save(sim_matrix_path, sim_matrix)
    
        
    # 2 : Smooth the similatry matrix
    print('Smoothing similarity matrix')
    sim_matrix_smoothed = smooth_surface_graph(graph, sim_matrix, iterations=5)

    # 3 : Compute the gradient of the similarty matrix
    print('Computing gradients')
    gradients = compute_gradients(graph, sim_matrix_smoothed)

    # 3.1 : Smooth the gradient
    print('Smoothing gradients')
    gradients_smoothed = smooth_surface_graph(graph, gradients, iterations=10)
    gradients_path = output_dir / f"gradients_smoothed_sub{subject}_fastway_run{run}_{hemisphere}.npy"
    np.save(gradients_path, gradients_smoothed)

    # 4 : Create the boundary map from the grad mean
    boundary_map = gradients_smoothed.mean(axis=1)
    
    boundary_map_path = output_dir / f"boundary_map_sub{subject}_fastway_run{run}_{hemisphere}.npy"
    np.save(boundary_map_path, boundary_map)
    # memory clean up
    del boundary_map, gradients, gradients_smoothed, sim_matrix, sim_matrix_smoothed

## Visualize the results of subject level parcellation

In [ ]:
# 5 : Visualize the bounardy map
config = {
"fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
"subjects_dir": Path(r"D:\Data_Conn_Preproc\PPSFACE_N18")
}
subject = f"{3:02d}"

surface_inf_path = config["fsavg6_dir"] / "surf" / f"lh.inflated"
surface_path = config["fsavg6_dir"] / "surf" / f"lh.white"

coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
coords, faces = nib.freesurfer.read_geometry(str(surface_path))
graph = build_mesh_graph(faces)

path_bound = f"D:\Data_Conn_Preproc\PPSFACE_N18\sub-{subject}\sub-{subject}_parcellation\boundary_map_sub{subject}_run1_lh.npy"
boundary_map = np.load(path_bound)
print('boundary_map shape', boundary_map.shape)


In [ ]:
visualize_brain_surface(coords_, faces_, boundary_map, title=f'Boundary map Subject n°{subject}')

In [ ]:
bound_smoothed = smooth_surface_graph(graph, boundary_map, iterations=10)
visualize_brain_surface(coords_, faces_, bound_smoothed)

In [ ]:
labels = watershed_by_flooding(graph, bound_smoothed)
visualize_brain_surface(coords_, faces_, labels, title=f'Parcellation of subject n°{subject}')